In [1]:
from typing import List, Tuple, Callable
import random
import numpy as np


class RepeatedPrisonersDilemma:
    """
    True: player cooperates
    False: player defects
    """
    def __init__(self):
        pass

    def run(self, n_repetitions: int, strategy_player_a: Callable, strategy_player_b: Callable) -> Tuple[Tuple[int, int], Tuple[List[bool], List[bool]]]:
        score_a = 0
        score_b = 0
        history_player_a = []
        history_player_b = []
        for _ in range(n_repetitions):
            turn_player_a = strategy_player_a(history_this_player=history_player_a, history_other_player=history_player_b)
            turn_player_b = strategy_player_b(history_this_player=history_player_b, history_other_player=history_player_a)
            turn_scores = self.score(player_a=turn_player_a, player_b=turn_player_b)
            score_a += turn_scores[0]
            score_b += turn_scores[1]
            history_player_a.append(turn_player_a)
            history_player_b.append(turn_player_b)
        return ((score_a, score_b), (history_player_a, history_player_b))

    @staticmethod
    def score(player_a: bool, player_b: bool) -> Tuple[int, int]:
        if player_a and player_b:
            return (3, 3)
        if player_a and not player_b:
            return (0, 6)
        if not player_a and player_b:
            return (6, 0)
        else:
            return (1, 1)

    @staticmethod
    def strategy_tit_for_tat(history_this_player: List[bool], history_other_player: List[bool]):
        """
        start with cooperation and afterwards exactly mirrors the last turn of the opponent
        """
        if len(history_this_player) == 0:
            return True
        return history_other_player[-1]
    
    @staticmethod
    def strategy_tit_for_tat_generous(history_this_player: List[bool], history_other_player: List[bool]):
        """
        tit for tat but with 10% chance forgives deflection of other player in last turn
        """
        if len(history_this_player) == 0:
            return True
        regular_turn = history_other_player[-1] 
        if regular_turn == False:
            return random.random() > 0.9
        return regular_turn

    @staticmethod
    def strategy_tit_for_tat_suspicious(history_this_player: List[bool], history_other_player: List[bool]):
        """
        tit for tat but starting with deflection (prefered in hostile environments)
        """
        if len(history_this_player) == 0:
            return False
        return history_other_player[-1]

    @staticmethod
    def strategy_tit_for_tat_noisy(history_this_player: List[bool], history_other_player: List[bool]):
        """
        randomly deviates from regular tit for tat turn with 10% chance 
        """
        if len(history_this_player) == 0:
            regular_turn = True
        else:
            regular_turn = history_other_player[-1]

        if random.random() > 0.9:
            return not regular_turn
        return regular_turn

    @staticmethod
    def strategy_tit_for_tat_exponential_decay(history_this_player: List[bool], history_other_player: List[bool]):
        """
        tit for tat but considering the history of the opponents actions, assigning more weight to recent moves.
        The player cooperates with a probability proportional to the weighted fraction of the opponents cooperation.
        """
        if len(history_this_player) == 0:
            return True
        weights = np.logspace(start=0.1, stop=1, base=10, num=len(history_other_player))
        return bool(random.random() < (weights[history_other_player].sum() / weights.sum()))

    @staticmethod
    def strategy_grim_trigger(history_this_player: List[bool], history_other_player: List[bool]):
        """
        starts with cooperation but always deflects once the other player deflects a single time
        """
        if sum(history_other_player) != len(history_other_player):
            return False
        return True

    @staticmethod
    def strategy_random(history_this_player: List[bool], history_other_player: List[bool]):
        """
        random deflection / cooperation
        """
        return random.random() > 0.5
    
    @staticmethod
    def strategy_naive_probability(history_this_player: List[bool], history_other_player: List[bool]):
        """
        starts with cooperation, then cooperates with same probability as the opponent 
        """
        if len(history_other_player) == 0:
            return True
        return random.random() <= sum(history_other_player) / len(history_other_player)




game = RepeatedPrisonersDilemma()
game.run(n_repetitions=10, strategy_player_a=game.strategy_random, strategy_player_b=game.strategy_tit_for_tat_exponential_decay)


((22, 22),
 ([False, False, False, False, False, True, True, True, True, False],
  [True, False, False, False, False, False, False, True, True, True]))